In [1]:
!pip install yfinance pandas numpy

#Create an agent to monitor stock prices under $1000 as investment recommendation

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class StockInvestmentAgent:
    def __init__(self, budget=1000):
        self.budget = budget
        self.stock_data = {}

    def get_stock_data(self, symbols, period='10d'):
        for symbol in symbols:
            stock = yf.Ticker(symbol)
            data = stock.history(period=period)
            if not data.empty:
                self.stock_data[symbol] = data

    def analyze_stocks(self):
        profitable_stocks = []
        for symbol, data in self.stock_data.items():
            closing_prices = data['Close']
            initial_price = closing_prices.iloc[0]
            final_price = closing_prices.iloc[-1]
            price_change = final_price - initial_price
            percentage_gain = (price_change / initial_price) * 100
            if initial_price <= self.budget:
                profitable_stocks.append((symbol, initial_price, percentage_gain))

        return sorted(profitable_stocks, key=lambda x: x[2], reverse=True)

    def recommend_investment(self):
        recommendations = self.analyze_stocks()
        if recommendations:
            best_stock = recommendations[0]
            symbol, price, gain = best_stock
            num_shares = self.budget // price
            investment = num_shares * price
            return {
                "Best Stock": symbol,
                "Price per Share": price,
                "Estimated Gain (%)": gain,
                "Total Investment": investment,
                "Number of Shares": num_shares
            }
        else:
            return {"Message": "No profitable investment found under $1000."}

symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA", "NFLX", "META"]
agent = StockInvestmentAgent(budget=1000)
agent.get_stock_data(symbols, period='5d')
recommendation = agent.recommend_investment()
recommendation

{'Best Stock': 'TSLA',
 'Price per Share': 191.75999450683594,
 'Estimated Gain (%)': 8.38027105995035,
 'Total Investment': 958.7999725341797,
 'Number of Shares': 5.0}

#Keep the stock monitoring agent running on background

In [6]:
!pip install schedule

  Using cached schedule-1.2.2-py3-none-any.whl.metadata (3.8 kB)


In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import schedule
import time
from datetime import datetime, timedelta

class StockInvestmentAgent:
    def __init__(self, budget=1000):
        self.budget = budget
        self.stock_data = {}

    def get_stock_data(self, symbols, period='1d'):
        for symbol in symbols:
            stock = yf.Ticker(symbol)
            data = stock.history(period=period)
            if not data.empty:
                self.stock_data[symbol] = data

    def analyze_stocks(self):
        profitable_stocks = []
        for symbol, data in self.stock_data.items():
            closing_prices = data['Close']
            initial_price = closing_prices.iloc[0]
            final_price = closing_prices.iloc[-1]
            price_change = final_price - initial_price
            percentage_gain = (price_change / initial_price) * 100
            if initial_price <= self.budget:
                profitable_stocks.append((symbol, initial_price, percentage_gain))

        return sorted(profitable_stocks, key=lambda x: x[2], reverse=True)

    def recommend_investment(self):
        recommendations = self.analyze_stocks()
        if recommendations:
            best_stock = recommendations[0]
            symbol, price, gain = best_stock
            num_shares = self.budget // price
            investment = num_shares * price
            print(f"Best investment: {symbol}")
            print(f"Price per share: ${price:.2f}")
            print(f"Estimated gain: {gain:.2f}%")
            print(f"Total investment: ${investment:.2f} for {num_shares} shares")
        else:
            print("No profitable investment found under $1000.")

def run_agent():
    symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA", "NFLX", "META"]
    agent = StockInvestmentAgent(budget=1000)
    agent.get_stock_data(symbols)
    agent.recommend_investment()

# Schedule the agent to run every day for 5 days
end_date = datetime.now() + timedelta(minutes=5)
schedule.every().day.at("10:00").do(run_agent)

while datetime.now() < end_date:
    schedule.run_pending()
    time.sleep(1)

print("The 5-day period is over. The agent has stopped running.")

The 5-day period is over. The agent has stopped running.
